# Create Hive SQL Files - create Tables, insert data for all metrics

In [1]:
from sqlalchemy import create_engine
from pyhive import hive

from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

engine = create_engine('hive://172.30.17.145:10000/default')

In [3]:
inspector = inspect(engine)

# Get column information
row=inspector.get_columns('tables_stage1_p_meta_complete')


col = [] 
for _row in row:
    col.append(_row['name'])

In [4]:
import pandas as pd
row =  engine.execute("select * from  tables_stage1_p_meta_complete")
df = pd.DataFrame(row)
df.columns = col
df

,product,service,__name__,alertname,alertstate,instance,job,severity,code,le,...,queue,env,state,logical_system,prio,port,drive,account,instanceid,queuename
0,BIS,CommunicationService,bis_jms_scheduled_count,None,None,clspromsg-edi01:13000,CBR,None,None,None,...,SAPConnectorClientComponent,None,None,None,None,None,None,None,None,None
1,BIS,CommunicationService,bis_order_execution_time_seconds_bucket,None,None,clspromsg-edi01:13000,CBR,None,None,0.75,...,None,None,None,None,None,None,None,None,None,None
2,BIS,CommunicationService,bis_order_sync_execution_total,None,None,clspromsg-edi01:13000,CBR,None,None,None,...,None,None,None,None,None,None,None,None,BISAS,None
3,BIS,CommunicationService,scrape_samples_post_metric_relabeling,None,None,clspromsg-edi02:13000,CBR,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,BIS,CommunicationService,bis_adapter_status,None,None,clspromsg-edi01:13000,CBR,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26681,Management,None,prometheus_target_sync_length_seconds,None,None,localhost:9090,prometheus,None,None,None,...,None,None,None,None,None,None,None,None,None,None
26682,Management,None,prometheus_target_scrape_pool_sync_total,None,None,localhost:9090,prometheus,None,None,None,...,None,None,None,None,None,None,None,None,None,None
26683,Management,None,prometheus_http_response_size_bytes_bucket,None,None,localhost:9090,prometheus,None,None,+Inf,...,None,None,None,None,None,None,None,None,None,None
26684,Management,None,prometheus_http_request_duration_seconds_bucket,None,None,localhost:9090,prometheus,None,None,120,...,None,None,None,None,None,None,None,None,None,None


In [148]:
def get_create(metric_table, location ) :
    create = "CREATE EXTERNAL TABLE" \
    + " " + metric_table \
    + ' (' \
    + ' `date` STRING, value STRING,' \
    + ' `product` STRING, `service` STRING, `__name__` STRING, `alertname` STRING, `alertstate` STRING, `instance` STRING, `job` STRING, `severity` STRING, `code` STRING, `le` STRING, `quantile` STRING, `scrape_job` STRING, `interval` STRING, `event` STRING, `role` STRING, `name` STRING, `config` STRING, `call` STRING, `endpoint` STRING, `rule_group` STRING, `handler` STRING, `slice` STRING, `branch` STRING, `goversion` STRING, `revision` STRING, `version` STRING, `listener_name` STRING, `dialer_name` STRING, `reason` STRING, `area` STRING, `runtime` STRING, `vendor` STRING,`gc` STRING,`pool` STRING,`cache` STRING,`adapterId` STRING,`adapter` STRING,`type` STRING,`engineType` STRING,`status` STRING,`remoteGroupId` STRING,`remoteInstanceId` STRING,`remoteSystemId` STRING,`queue` STRING,`env` STRING,`state` STRING,`logical_system` STRING,`prio` STRING,`port` STRING,`drive` STRING,`account` STRING,`instanceId` STRING,`queueName` STRING'\
    + ' ,schemaid BIGINT)' \
    + ' STORED as PARQUET' \
    + ' LOCATION ' + '\'' + location + '\'' + ';' 
    return create

def get_insert(metric_table, data_table, where ) :
    insert = 'INSERT INTO TABLE ' + metric_table \
    + ' select ' \
    + ' `date`, value,' \
    + '`product` , `service` , `__name__` , `alertname` , `alertstate` , `instance` , `job` , `severity` , `code` , `le` , `quantile` , `scrape_job` , `interval` , `event` , `role` , `name` , `config` , `call` , `endpoint` , `rule_group` , `handler` , `slice` , `branch` , `goversion` , `revision` , `version` , `listener_name` , `dialer_name` , `reason` , `area` , `runtime` , `vendor` ,`gc` ,`pool` ,`cache` ,`adapterId` ,`adapter` ,`type` ,`engineType` ,`status` ,`remoteGroupId` ,`remoteInstanceId` ,`remoteSystemId` ,`queue` ,`env` ,`state` ,`logical_system` ,`prio` ,`port` ,`drive` ,`account` ,`instanceId` ,`queueName`' \
    + ', `schemaid`' \
    + ' from ' + data_table \
    + ' where' + ''.join(where) \
    + 'order by `date`' \
    + ';'
    return insert

def get_drop(table) :
    return 'drop table if exists ' + table + ';';

def get_where(index):
    _where = []
    for x in df.columns :
        if(df.iloc[index][df.columns.get_loc(x)]) :
            #print(x,df.iloc[index][df.columns.get_loc(x)])
            if len(_where) > 0 :
                val = ' and ' + "`" + x +  "`" + ' = ' + '\'' + df.iloc[index][df.columns.get_loc(x)  ] + '\''
                _where.append(val)
            else:
                val = ' ' +  "`" + x +  "`" + ' = ' + '\'' + df.iloc[index][df.columns.get_loc(x)  ] + '\''
                _where.append(val)
    return _where
                
def writeSqlToFile(text_file, metric_table, location, data_table, where):
    text_file.write("%s\n" % get_drop(metric_table))
    text_file.write("%s\n" % get_create(metric_table, location ))
    text_file.write("%s\n" % get_insert(metric_table, data_table, where))
   
    

In [151]:
text_file = open("createMetricsTables.sql", "w")

for i in df.index :
    where = get_where(i)  
    data_table =  'tables_stage1_p_flat'           
    metric_table = 'metric_' + str(i) 
    location = '/user/admin/metrics/' + metric_table
    writeSqlToFile(text_file, metric_table, location, data_table, where)
    
text_file.close()

# Testing

In [146]:
df.columns
len(df.index)
df.iloc[1]
len(df.columns)
df.columns[10]
print(df.columns[3],df.iloc[2][df.columns[3]])
get_where(0)
#df.columns
len(df.index)

alertname None


26686